In [1]:
import time
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from sqlalchemy import create_engine

In [2]:


options = Options()
options.add_argument("--start-maximized")
options.add_argument("--headless=new")  # Ejecuta en segundo plano (modo headless)
options.add_argument("--disable-gpu")   # Opcional: evita errores gráficos en algunas máquinas
options.add_argument("--no-sandbox")    # Útil si corres esto en entornos tipo servidor


driver = webdriver.Chrome(service=Service(), options=options)
driver.get("https://www.waze.com/es-419/live-map")
time.sleep(20)

wait = WebDriverWait(driver, 10)
data = []

try:
    acknowledge_button = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, "waze-tour-tooltip__acknowledge")))
    acknowledge_button.click()
except Exception as e:
    print("No se pudo hacer clic en el botón 'Entendido':", e)
    
# Extraer baches y sus z-index
baches_data = []
baches = driver.find_elements(By.CLASS_NAME, "wm-alert-cluster-icon--hazard-on-road-pot-hole")
for b in baches:
    style = b.get_attribute("style")
    z = None
    for part in style.split(";"):
        if "z-index" in part:
            try:
                z = int(part.split(":")[1].strip())
            except:
                z = None
            break
    if z is not None:
        baches_data.append({"z": z, "style": style})

# Ordenar de mayor a menor z-index
baches_data.sort(key=lambda x: x["z"], reverse=True)

for i, bache_info in enumerate(baches_data, 1):
    try:
        print(f"➡ Procesando bache #{i} con z-index {bache_info['z']}")

        # Buscar el bache con ese z-index (de nuevo, para evitar stale element)
        baches_actuales = driver.find_elements(By.CLASS_NAME, "wm-alert-cluster-icon--hazard-on-road-pot-hole")
        bache = None
        for b in baches_actuales:
            if str(bache_info["z"]) in b.get_attribute("style"):
                bache = b
                break

        if not bache:
            print("⚠️ Bache no encontrado, se omite.")
            continue

        driver.execute_script("arguments[0].scrollIntoView(true);", bache)
        time.sleep(2)
        bache.click()
        time.sleep(2)

        tiempo_element = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "wm-alert-details__time")))
        tiempo_texto = tiempo_element.text.strip()

        like_element = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "wm-alert-metrics__likes")))
        like_texto = like_element.text.strip()

        print("📆 Tiempo:", tiempo_texto)
        print("👥 Reportado por:", like_texto)

        # Cerrar el popup
        cerrar_popup = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, "leaflet-popup-close-button")))
        cerrar_popup.click()
        time.sleep(2)

        # Buscar el mismo bache nuevamente antes del clic derecho
        baches_actuales = driver.find_elements(By.CLASS_NAME, "wm-alert-cluster-icon--hazard-on-road-pot-hole")
        bache = None
        for b in baches_actuales:
            if str(bache_info["z"]) in b.get_attribute("style"):
                bache = b
                break

        if not bache:
            print("⚠️ Bache desapareció antes de clic derecho.")
            continue

        driver.execute_script("arguments[0].scrollIntoView(true);", bache)
        time.sleep(2)
        ActionChains(driver).move_to_element(bache).context_click().perform()
        time.sleep(2)

        compartir_btn = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Compartir')]")))
        compartir_btn.click()
        time.sleep(2)

        boton_copiar = wait.until(EC.presence_of_element_located((By.XPATH, "//button[@data-ga-action='copy_drive_link']")))
        input_box = boton_copiar.find_element(By.XPATH, "preceding-sibling::input")
        link = input_box.get_attribute("value")

        print("✅ Link:", link)
        data.append([link, tiempo_texto, like_texto])

        cerrar_btn = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, "js-close")))
        cerrar_btn.click()
        time.sleep(1)

    except Exception as e:
        print(f"⚠️ Error en bache #{i}: {e}")
        driver.save_screenshot(f"error_bache_{i}.png")
        continue

# Guardar en CSV
with open("baches.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["Link", "Tiempo", "Likes"])
    for row in data:
        writer.writerow(row)

print("✅ Todo listo. Datos guardados en 'baches.csv'.")


➡ Procesando bache #1 con z-index 1017
📆 Tiempo: Último reporte hace 5 días
👥 Reportado por: 2
✅ Link: https://ul.waze.com/ul?ll=21.13990815%2C-86.89619494&navigate=yes&zoom=14&utm_campaign=default&utm_source=waze_website&utm_medium=lm_share_location
➡ Procesando bache #2 con z-index 1009
📆 Tiempo: Último reporte hace 5 días
👥 Reportado por: 20
✅ Link: https://ul.waze.com/ul?ll=21.14054858%2C-86.88623954&navigate=yes&zoom=14&utm_campaign=default&utm_source=waze_website&utm_medium=lm_share_location
➡ Procesando bache #3 con z-index 974
📆 Tiempo: Último reporte hace 4h 38 min
👥 Reportado por: 21
✅ Link: https://ul.waze.com/ul?ll=21.14335045%2C-86.86976162&navigate=yes&zoom=14&utm_campaign=default&utm_source=waze_website&utm_medium=lm_share_location
➡ Procesando bache #4 con z-index 928
📆 Tiempo: Último reporte hace 1h 29 min
👥 Reportado por: 1
✅ Link: https://ul.waze.com/ul?ll=21.14703283%2C-86.84693285&navigate=yes&zoom=14&utm_campaign=default&utm_source=waze_website&utm_medium=lm_share

InvalidSessionIdException: Message: invalid session id
Stacktrace:
	GetHandleVerifier [0x00007FF7EEC5EFA5+77893]
	GetHandleVerifier [0x00007FF7EEC5F000+77984]
	(No symbol) [0x00007FF7EEA28FEC]
	(No symbol) [0x00007FF7EEA6F44F]
	(No symbol) [0x00007FF7EEAA6FF2]
	(No symbol) [0x00007FF7EEAA1A02]
	(No symbol) [0x00007FF7EEAA0AC9]
	(No symbol) [0x00007FF7EE9F5AB5]
	GetHandleVerifier [0x00007FF7EEF17B6D+2931725]
	GetHandleVerifier [0x00007FF7EEF12132+2908626]
	GetHandleVerifier [0x00007FF7EEF300F3+3031443]
	GetHandleVerifier [0x00007FF7EEC791EA+184970]
	GetHandleVerifier [0x00007FF7EEC8086F+215311]
	(No symbol) [0x00007FF7EE9F4B2C]
	GetHandleVerifier [0x00007FF7EF022518+4023736]
	BaseThreadInitThunk [0x00007FFE5ABCE8D7+23]
	RtlUserThreadStart [0x00007FFE5C7914FC+44]


In [ ]:
import pandas as pd
from datetime import datetime, timedelta
import re

# Ejemplo de datos
df = pd.DataFrame({
    'comentario': [
        'Último reporte hace 1 día',
        'Último reporte hace 14h 4 min',
        'Último reporte hace 5 días',
        'Último reporte hace 22h 27 min'
    ]
})

# Función para restar tiempo al datetime actual
def obtener_fecha_reporte(texto):
    ahora = datetime.now()
    
    # Extraer días
    dias = int(re.search(r'(\d+)\s*d[ií]as?', texto).group(1)) if re.search(r'(\d+)\s*d[ií]as?', texto) else 0
    
    # Extraer horas
    horas = int(re.search(r'(\d+)\s*h', texto).group(1)) if re.search(r'(\d+)\s*h', texto) else 0
    
    # Extraer minutos
    minutos = int(re.search(r'(\d+)\s*min', texto).group(1)) if re.search(r'(\d+)\s*min', texto) else 0
    
    # Calcular fecha del reporte
    return ahora - timedelta(days=dias, hours=horas, minutes=minutos)




In [ ]:
import pandas as pd
import openpyxl

In [ ]:
df = pd.DataFrame(data)

In [ ]:
df['latitud'] = df.iloc[:, 0].str[26:37]

# Extraer caracteres 41 al 53 (posición 40 a 53)
df['longitud'] = df.iloc[:, 0].str[40:52]

In [ ]:
# Aplicar al DataFrame
df['fecha_reporte'] = df[1].apply(obtener_fecha_reporte)

In [ ]:
df.columns = ['URL', 'Reporte', 'Likes', 'latitud','longitud' ,'fecha_reporte' ]

In [ ]:
df.to_excel('baches.xlsx', sheet_name='baches', index=False)

In [ ]:
# Datos de conexión
usuario = 'juanyam'        # ← Reemplaza esto
contrasena = 'eJnNPmklNznIkZ1EJ8JB4B=='  # ← Reemplaza esto
host = '45.132.241.118'
puerto = '5432'
base_de_datos = 'siginplan'
esquema = 'datos_trafico'
nombre_tabla = 'baches'  # ← Nombre de la nueva tabla

# Crear conexión con SQLAlchemy
engine = create_engine(f'postgresql+psycopg2://{usuario}:{contrasena}@{host}:{puerto}/{base_de_datos}')

# Subir el DataFrame a PostgreSQL (al esquema deseado)
df.to_sql(nombre_tabla, engine, schema=esquema, if_exists='replace', index=False)
